In [1]:
import numpy as np
import cv2
import pandas as pd
import functools
import os

In [ ]:
help(cv2.calibrateCamera)

In [2]:
def draw_line (img, rho, theta, color=(0,255,0)):
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(np.around(x0 + 1000*(-b)))   # Here i have used int() instead of rounding the decimal value, so 3.8 --> 3
    y1 = int(np.around(y0 + 1000*(a)))    # But if you want to round the number, then use np.around() function, then 3.8 --> 4.0
    x2 = int(np.around(x0 - 1000*(-b)))   # But we need integers, so use int() function after that, ie int(np.around(x))
    y2 = int(np.around(y0 - 1000*(a)))
    cv2.line(img,(x1,y1),(x2,y2),color,2)
    #print (rho, theta, color)

# Given y, this function returns x from (rho, theta) line equation
def x_from_y(rho, theta, y):
    sin_t = np.sin(theta)
    cos_t = np.cos(theta) 

    return ((rho / cos_t) - ((y * sin_t) / cos_t))

# Given x, this function returns y from (rho, theta) line equation
def y_from_x(rho, theta, x):
    sin_t = np.sin(theta)
    cos_t = np.cos(theta) 

    return (-((x * cos_t) / sin_t) + (rho / sin_t)) 

# Given a point and the rectangle, this function checks 
# if this point lies inside the rectangle
def check_point(x, y, left_x, right_x, up_y, down_y):
    if (left_x <= x) and (x <= right_x) and (up_y <= y) and (y <= down_y):
        return (int(np.around(x)), int(np.around(y)))
    else:
        return None

# A line which traverses some rectangle has two points of traversal.
#
# This function finds four possible traversal points and drops those of them 
# which are not inside the rectangle
def get_traversal_points(rho, theta, left_x, right_x, up_y, down_y):      

    #Find four possible points of traversal
    up = (x_from_y(rho, theta, up_y), up_y)
    right = (right_x, y_from_x(rho, theta, right_x))
    down = (x_from_y(rho, theta, down_y), down_y)
    left = (left_x, y_from_x(rho, theta, left_x))

    #If point is inside, we add it to the list; otherwise we add None
    intersect_points = list(map(lambda p: check_point(p[0], p[1], left_x, right_x, up_y, down_y), [up, right, down, left]))

    #Drop all None values
    #intersect_points = [point for point in intersect_points if point != None]
    intersect_points = [down, up]

    #Check if exactly two points were found
    if len(intersect_points) != 2:
        print('Error! len(intersect_points) != 2, intersect_points = {}'.format(intersect_points))
        return None
    else:
        return intersect_points


# Get the point of traversal for 'plus' and 'minus' lines
# The order as follows in a way to obtain convenient trapezoid
def get_vanishing_point(rho_1, theta_1, rho_2, theta_2):
    k_1 = - np.cos(theta_1) / np.sin(theta_1)
    m_1 = rho_1 / np.sin(theta_1)

    k_2 = - np.cos(theta_2) / np.sin(theta_2)
    m_2 = rho_2 / np.sin(theta_2)

    x = (m_1 - m_2) / (k_1 - k_2)
    y = (k_1 * m_2 - k_2 * m_1) / (k_1 - k_2)

    return (x, y)

In [5]:
test_points = 0
def CorrectPerspectiveDistortion (input_folder, filename, output_folder):
    #Read the image
    base = input_folder
    im = cv2.imread(base + filename)
    gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    equ = cv2.equalizeHist(gray)
    #ret, thresh = cv2.threshold(equ,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    winSize = 3
    blured = cv2.GaussianBlur(equ, (winSize, winSize), 0)
    edges = cv2.Canny(blured,100,220,apertureSize = 3, L2gradient=True)

    cv2.imwrite(output_folder+'gray_'+filename, gray)
    cv2.imwrite(output_folder+'equ_'+filename, equ)
    cv2.imwrite(output_folder+'blured_'+filename, blured)
    cv2.imwrite(output_folder+'edges_'+filename, edges)

    height = im.shape[0]
    width = im.shape[1]

    img = im.copy()
    rho_minus = 0
    rho_plus = np.inf

    eps = 0.01
    lines = cv2.HoughLines(edges,1,np.pi/180 , int (min(im.shape[0], im.shape[1]) / 3.25))
    #print (len(lines))
    
    red = (0, 0, 255)
    
    for line in lines:
        for rho,theta in line:
            draw_line (img, rho, theta)
            if (theta > np.pi / 2 + eps and np.abs (rho) > np.abs (rho_minus)):
                rho_minus = rho
                theta_minus = theta
            if (theta < np.pi / 2 - eps and rho < rho_plus):
                rho_plus = rho
                theta_plus = theta

    
    draw_line (img, rho_minus, theta_minus, red)
    draw_line (img, rho_plus, theta_plus, red)

    cv2.imwrite(output_folder+'houghlines_red._'+filename, img)


    van_x, van_y = get_vanishing_point(rho_plus, theta_plus, rho_minus, theta_minus)

    offset = 0
    if van_y > 0:
        low_y = van_y + 15 #A small offset to avoid infinite width of the vanishing point
        offset = low_y
    else:
        low_y = 0

    plus_down, plus_up = get_traversal_points(rho_plus, theta_plus, 0, width, low_y, height)
    minus_down, minus_up = get_traversal_points(rho_minus, theta_minus, 0, width, low_y, height)

    # Create a transform and apply it
    from_points = np.float32([plus_down, plus_up, minus_up, minus_down])
    init_width = int(np.around(minus_down[0] - plus_down[0]))

    plus_down = (plus_down[0] + max(0, -plus_down[0]), plus_down[1])
    minus_down = (plus_down[0] + init_width, minus_down[1])

    to_points = np.float32([plus_down, (plus_down[0], plus_up[1]), (minus_down[0], minus_up[1]), minus_down])

    M = cv2.getPerspectiveTransform(from_points, to_points)
    input_corners = np.array([[(0, height), (0, plus_up[1]), (width, minus_up[1]), (width, height)]], dtype=float)

    output_corners = cv2.perspectiveTransform(input_corners, M)

    x_min = functools.reduce(lambda x,y: [min(x[0], y[0]), 0], output_corners[0])[0]
    x_max = functools.reduce(lambda x,y: [max(x[0], y[0]), 0], output_corners[0])[0]
    y_min = functools.reduce(lambda x,y: [0, min(x[1], y[1])], output_corners[0])[1]
    y_max = functools.reduce(lambda x,y: [0, max(x[1], y[1])], output_corners[0])[1]

    br_x = x_min
    br_y = y_min

    new_width = int(x_max - x_min)
    new_height = int(y_max - y_min)

    to_points = np.float32([(to_point[0] - br_x, to_point[1] - br_y) for to_point in to_points])
    M_shifted = cv2.getPerspectiveTransform(from_points, to_points)

    dst = cv2.warpPerspective(im, M_shifted, (new_width, new_height))
    cv2.imwrite(output_folder+'transformed_'+filename, dst)
    return from_points, im

In [6]:
test_points = 0
for file in os.listdir('test_images'):
    test_points, img = CorrectPerspectiveDistortion ('test_images/', file, 'output_images/')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:65: RuntimeWarning: divide by zero encountered in float_scalars
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:66: RuntimeWarning: divide by zero encountered in float_scalars
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:71: RuntimeWarning: invalid value encountered in float_scalars
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: invalid value encountered in float_scalars
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in float_scalars
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in double_scalars


error: OpenCV(3.4.1) /Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/color.cpp:11147: error: (-215) scn == 3 || scn == 4 in function cvtColor


In [ ]:
t = test_points
A = np.array([
             [t[0][0], t[0][1], 0,       0,       t[0][0]**2,      t[0][0]*t[0][1]],
             [t[1][0], t[1][1], 0,       0,       t[0][0]*t[1][0], t[0][0]*t[1][1]],
             [t[2][0], t[1][1], 0,       0,       t[2][0]*t[3][0], t[1][1]*t[3][0]],
             [t[3][0], t[0][1], 0,       0,       t[3][0]**2,      t[0][1]*t[3][0]],
             [0,       0,       t[0][0], t[0][1], t[0][0]*t[0][1], t[0][1]**2     ],
             [0,       0,       t[3][0], t[0][1], t[3][0]*t[0][1], t[0][1]**2     ]
            ])
b = np.array([[t[0][0]], [t[0][0]], [t[3][0]], [t[3][0]], [t[0][1]], [t[0][1]]])

x0 = np.dot (np.linalg.inv(A), b).T[0]
print (x0)

M = np.array([[x0[0], x0[1], 0],
               [x0[2], x0[3], 0],
               [x0[4], x0[5], 1]
              ])

out = cv2.warpPerspective(img, M, (img.shape[0], img.shape[1]))
cv2.imwrite('hi.png', out)

## Original code is below

In [14]:
# help(cv2.threshold)

#Read the image
base = './test_images/'
im = cv2.imread(base + '3.jpg')
gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
equ = cv2.equalizeHist(gray)
#ret, thresh = cv2.threshold(equ,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
winSize = 3
blured = cv2.GaussianBlur(equ, (winSize, winSize), 0)
edges = cv2.Canny(blured,100,220,apertureSize = 3, L2gradient=True)



#cv2.imshow('houghlines',im)
cv2.imwrite(base+'gray.jpg', gray)
cv2.imwrite(base+'equ.jpg', equ)
cv2.imwrite(base+'blured.jpg', blured)
cv2.imwrite(base+'edges.jpg', edges)

#cv2.imwrite(base+'thresh.jpg', thresh)

#A few changes to check git functionality
print(im.shape)

height = im.shape[0]
width = im.shape[1]


img = im.copy()
rho_minus = 0
rho_plus = np.inf

rho_hor_max = 0
hor_max_found = False
rho_hor_min = im.shape[0]
hor_min_found = False
eps = 0.01
lines = cv2.HoughLines(edges,1,np.pi/180, int (min(im.shape[0], im.shape[1]) / 3.25))
#print (lines)
for line in lines:
    for rho,theta in line:
        if (theta > np.pi / 2 + eps and np.abs (rho) > np.abs (rho_minus)):
            rho_minus = rho
            theta_minus = theta
        if (theta < np.pi / 2 - eps and rho < rho_plus):
            rho_plus = rho
            theta_plus = theta
        if (np.abs (theta - np.pi / 2) < eps):
            if (rho > rho_hor_max):
                rho_hor_max = rho
                hor_max_found = True
            if (rho < rho_hor_min):
                rho_hor_min = rho
                hor_min_found = True
        #draw_line (img, rho, theta)
        
red = (0, 0, 255)
draw_line (img, rho_minus, theta_minus, red)
draw_line (img, rho_plus, theta_plus, red)

cv2.imwrite(base+'houghlines_red.jpg', img)


# Get the point of traversal for 'plus' and 'minus' lines
# The order as follows in a way to obtain convenient trapezoid

van_x, van_y = get_vanishing_point(rho_plus, theta_plus, rho_minus, theta_minus)

offset = 0
if van_y > 0:
    low_y = van_y + 15#(height - van_y) // 15
    offset = low_y
else:
    low_y = 0
    print('I am a good boy')

plus_down, plus_up = get_traversal_points(rho_plus, theta_plus, 0, width, low_y, height)
minus_down, minus_up = get_traversal_points(rho_minus, theta_minus, 0, width, low_y, height)

print(plus_down, plus_up)
print(minus_down, minus_up)

# strip = im[:, plus_up[0]:minus_up[0]]
# cv2.imwrite(base + 'strip.jpg', strip)

#strip_edges = edges[:, plus_up[0]:minus_up[0]]

#strip_img = strip.copy()

# rho_hor_min = strip.shape[0]
# hor_min_found = False
# eps = 0.01
# lines = cv2.HoughLines(strip_edges,1,np.pi/180, int (min(strip.shape[0], strip.shape[1]) / 2))

# for line in lines:
#     for rho,theta in line:
#         if (np.abs (theta - np.pi / 2) < eps):
#             if (rho < rho_hor_min):
#                 rho_hor_min = rho
#                 hor_min_found = True
#         #draw_line (img, rho, theta)

# if hor_min_found:
#     draw_line (strip_img, rho_hor_min, np.pi / 2, red)
# else:
#     print('No roof was found!')
#     rho_hor_min = 0
    
# cv2.imwrite(base+'strip_houghlines_red.jpg', strip_img)

# crop_up = int(np.around(rho_hor_min))
# crop_down = min(minus_down[1], plus_down[1])
# print(crop_up, crop_down)

# crop = im[crop_up:crop_down, :]
# cv2.imwrite(base + 'cropped.jpg', crop)

# Get the points of traversal for the cropped picture
# crop_plus_down, crop_plus_up = get_traversal_points(rho_plus, theta_plus, 0, width, crop_up, crop_down)
# crop_minus_up, crop_minus_down = get_traversal_points(rho_minus, theta_minus, 0, width, crop_up, crop_down)

# crop_height, crop_width = crop.shape[0], crop.shape[1]

# Create a transform and apply it
from_points = np.float32([plus_down, plus_up, minus_up, minus_down])

init_width = int(np.around(minus_down[0] - plus_down[0]))

plus_down = (plus_down[0] + max(0, -plus_down[0]), plus_down[1])
minus_down = (plus_down[0] + init_width, minus_down[1])

to_points = np.float32([plus_down, (plus_down[0], plus_up[1]), (minus_down[0], minus_up[1]), minus_down])

(430, 275, 3)
I am a good boy
(25.530525395041032, 430) (48.06587219238281, 0)
(246.84278885955155, 430) (224.30740356445312, 0)


In [10]:
height

876

In [16]:
f = np.sqrt(width**2+height**2) * 2.
print (van_y)
is_dist = height/ 2 - van_y
print (f, is_dist)
sin_alpha = f / np.sqrt(f**2 + is_dist**2)
cos_alpha = is_dist / np.sqrt(f**2 + is_dist**2)
tilt_matrix = np.array([[1, 0, 0],
                        [0, cos_alpha, -sin_alpha],
                        [0, sin_alpha, cos_alpha]])
shift_matrix = np.array([[1, 0, -width / 2 / f],
                         [0, 1, 0],
                         [0, 0, 1]])
M = np.dot(tilt_matrix, shift_matrix)
print (plus_down, plus_up)
start = [
    [-width/2, height, f],
    [-width/2, 0, f],
    #[plus_up[0], plus_up[1], f],
    #[plus_down[0], plus_down[1], f],
    [width/2, 0, f],
    [width/2, height, f]
]
print (start)
t_st = np.float32([(x[0],x[1]) for x in start])
finish = []
for x in start:
    tmp = np.dot(M, np.transpose(x))
    finish.append ((tmp[0]*f/tmp[2], tmp[1]*f/tmp[2]))
print (finish)

matr = cv2.getPerspectiveTransform(t_st, np.array([finish], dtype=np.float32))

x_min = functools.reduce(lambda x,y: [min(x[0], y[0]), 0], finish)[0]
x_max = functools.reduce(lambda x,y: [max(x[0], y[0]), 0], finish)[0]
y_min = functools.reduce(lambda x,y: [0, min(x[1], y[1])], finish)[1]
y_max = functools.reduce(lambda x,y: [0, max(x[1], y[1])], finish)[1]

br_x = x_min / 2
br_y = y_min

new_width = int(x_max - x_min)
new_height = int(y_max - y_min)

to_points = np.float32([(to_point[0] - br_x, to_point[1] - br_y) for to_point in finish])
M_shifted = cv2.getPerspectiveTransform(t_st, to_points)

dst = cv2.warpPerspective(im, M_shifted, (new_width, new_height))
cv2.imwrite(base+'transformed1.jpg', dst)

-1681.4429
1020.8329931972222 1896.44287109375
(25.530525395041032, 430) (48.06587219238281, 0)
[[-137.5, 430, 1020.8329931972222], [-137.5, 0, 1020.8329931972222], [137.5, 0, 1020.8329931972222], [137.5, 430, 1020.8329931972222]]
[(-254.58543610599241, -97.41462738912652), (-312.31021083777233, -549.5024479165995), (-8.589324082316534e-15, -549.5024479165995), (-7.0017461532442e-15, -97.41462738912652)]


True

# My point of view

In [15]:
f = np.sqrt(width**2+height**2)
print (van_y)
van_dist = abs(- height/ 2 + van_y)
print (f, van_dist)

theta = np.arctan(van_dist / f)
cos_t = np.cos(theta)
sin_t = np.sin(theta)
tg_t = np.tan(theta)

raw_from_points = [(0, 0), (0, f / tg_t), (f / sin_t, f / tg_t), (f / (2 * sin_t), f / np.tan(2 * theta))]
raw_to_points = [(0, -f / tg_t), (0, 0), (f, 0), (f, -f * tg_t)]

from_points = np.float32([(x + (width / 2), y + (height / 2)) for x, y in raw_from_points])
to_points = np.float32([(x + (width / 2), y + (height / 2)) for x, y in raw_to_points])

M = cv2.getPerspectiveTransform(from_points, to_points)

input_corners = np.array([[(0, height), (0, 0), (width, 0), (width, height)]], dtype=float)

output_corners = cv2.perspectiveTransform(input_corners, M)
print(output_corners)


x_min = functools.reduce(lambda x,y: [min(x[0], y[0]), 0], output_corners[0])[0]
x_max = functools.reduce(lambda x,y: [max(x[0], y[0]), 0], output_corners[0])[0]
y_min = functools.reduce(lambda x,y: [0, min(x[1], y[1])], output_corners[0])[1]
y_max = functools.reduce(lambda x,y: [0, max(x[1], y[1])], output_corners[0])[1]

br_x = x_min
br_y = y_min

new_width = int(x_max - x_min)
new_height = int(y_max - y_min)

to_points = np.float32([(to_point[0] - br_x, to_point[1] - br_y) for to_point in to_points])
M_shifted = cv2.getPerspectiveTransform(from_points, to_points)

dst = cv2.warpPerspective(im, M_shifted, (new_width, new_height))
cv2.imwrite('./My_transformed.jpg', dst)

# sin_alpha = f / np.sqrt(f**2 + is_dist**2)
# cos_alpha = is_dist / np.sqrt(f**2 + is_dist**2)
# tilt_matrix = np.array([[1, 0, 0],
#                         [0, cos_alpha, -sin_alpha],
#                         [0, sin_alpha, cos_alpha]])
# shift_matrix = np.array([[1, 0, -width / 2 / f],
#                          [0, 1, 0],
#                          [0, 0, 1]])
# M = np.dot(tilt_matrix, shift_matrix)
# print (plus_down, plus_up)
# start = [
#     [-width/2, height, f],
#     [-width/2, 0, f],
#     #[plus_up[0], plus_up[1], f],
#     #[plus_down[0], plus_down[1], f],
#     [width/2, 0, f],
#     [width/2, height, f]
# ]
# print (start)
# t_st = np.float32([(x[0],x[1]) for x in start])
# finish = []
# for x in start:
#     tmp = np.dot(M, np.transpose(x))
#     finish.append ((tmp[0]*f/tmp[2], tmp[1]*f/tmp[2]))
# print (finish)

# matr = cv2.getPerspectiveTransform(t_st, np.array([finish], dtype=np.float32))

# x_min = functools.reduce(lambda x,y: [min(x[0], y[0]), 0], finish)[0]
# x_max = functools.reduce(lambda x,y: [max(x[0], y[0]), 0], finish)[0]
# y_min = functools.reduce(lambda x,y: [0, min(x[1], y[1])], finish)[1]
# y_max = functools.reduce(lambda x,y: [0, max(x[1], y[1])], finish)[1]

# br_x = x_min / 2
# br_y = y_min

# new_width = int(x_max - x_min)
# new_height = int(y_max - y_min)

# to_points = np.float32([(to_point[0] - br_x, to_point[1] - br_y) for to_point in finish])
# M_shifted = cv2.getPerspectiveTransform(t_st, to_points)

# dst = cv2.warpPerspective(im, M_shifted, (new_width, new_height))
# cv2.imwrite(base+'transformed1.jpg', dst)

-1681.4429
510.4164965986111 1896.44287109375
[[[   9.60623548  284.72020175]
  [ -23.10038307 -182.4325998 ]
  [ 298.10038714 -182.43260114]
  [ 265.39376206  284.72020194]]]


True

In [ ]:
sin_t, cos_t, tg_t

In [ ]:
t = from_points
A = np.array([
             [t[0][0], t[0][1], 0,       0,       -t[0][0]**2,      -t[0][0]*t[0][1]],
             [t[1][0], t[1][1], 0,       0,       -t[0][0]*t[1][0], -t[0][0]*t[1][1]],
             [t[2][0], t[1][1], 0,       0,       -t[2][0]*t[3][0], -t[1][1]*t[3][0]],
             [t[3][0], t[0][1], 0,       0,       -t[3][0]**2,      -t[0][1]*t[3][0]],
             [0,       0,       t[0][0], t[0][1], -t[0][0]*t[0][1], -t[0][1]**2     ],
             [0,       0,       t[3][0], t[0][1], -t[3][0]*t[0][1], -t[0][1]**2     ]
            ])
b = np.array([[t[0][0]], [t[0][0]], [t[3][0]], [t[3][0]], [t[0][1]], [t[0][1]]])

x0 = np.dot (np.linalg.inv(A), b).T[0]
print (x0)

M = np.array([[x0[0], x0[1], 0],
               [x0[2], x0[3], 0],
               [x0[4], x0[5], 1]
              ])

In [ ]:
#M = cv2.getPerspectiveTransform(from_points, to_points)
input_corners = np.array([[(0, height), (0, plus_up[1]), (width, minus_up[1]), (width, height)]], dtype=float)

output_corners = cv2.perspectiveTransform(input_corners, M)
print (output_corners)
x_min = functools.reduce(lambda x,y: [min(x[0], y[0]), 0], output_corners[0])[0]
x_max = functools.reduce(lambda x,y: [max(x[0], y[0]), 0], output_corners[0])[0]
y_min = functools.reduce(lambda x,y: [0, min(x[1], y[1])], output_corners[0])[1]
y_max = functools.reduce(lambda x,y: [0, max(x[1], y[1])], output_corners[0])[1]

br_x = x_min
br_y = y_min

shift = np.array ([[1, 0, -br_x],
                   [0, 1, -br_y],
                   [0, 0, 1]])
print (shift)

new_width = int(x_max - x_min)
new_height = int(y_max - y_min)

#to_points = np.float32([(to_point[0] - br_x, to_point[1] - br_y) for to_point in to_points])
M_shifted = np.dot (shift, M)

#dst = cv2.warpPerspective(im, M_shifted, (max(init_width, width), height))
dst = cv2.warpPerspective(im, M_shifted,  (new_width, new_height))
print (dst.shape)
cv2.imwrite(base+'transformed1.jpg', dst)

In [ ]:
help(cv2.boundingRect)
cv2.boundi

In [ ]:
x_min, x_max

In [ ]:
x1

In [ ]:
def get_vanishing_point(rho_1, theta_1, rho_2, theta_2):
    k_1 = - np.cos(theta_1) / np.sin(theta_1)
    m_1 = rho_1 / np.sin(theta_1)
    
    k_2 = - np.cos(theta_2) / np.sin(theta_2)
    m_2 = rho_2 / np.sin(theta_2)
    
    x = (m_1 - m_2) / (k_1 - k_2)
    y = (k_1 * m_2 - k_2 * m_1) / (k_1 - k_2)
    
    return (x, y)
